In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Modeling/
!ls

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1vvRy60J06fZq-dj8EPVTkgHU4gq8OCqU/Capstone-KPMG/Modeling
 all_route_10mile_adjacent.pkl	 Optimization		  'Transfer Learning'
 Classification			 optimization_result.csv
 file.pkl			 OptimizationV2.ipynb


In [ ]:
import pandas as pd
import numpy as np
from map_request_api import *

ModuleNotFoundError: ignored

In [ ]:
# Load attractions data
attractions = pd.read_csv('Data/attraction_point.csv')
# covert attractions to remoteData Format
attractionRomoteDataList = getRemoteDataList(attractions, 'Y', 'X', 'name')

In [ ]:
# Load gas stations
gas_stations_all = pd.read_csv('Data/wa_gas_stations.csv')
gas_stations_all

,Longitude,Latitude,name
0,-122.329806,48.393760,24 Hour Express Fuel
1,-120.916225,47.191712,76
2,-122.357577,47.238704,76
3,-122.291324,48.435227,76
4,-120.014523,47.839608,76
...,...,...,...
2206,-122.127847,48.198460,NaN
2207,-122.344545,48.556192,NaN
2208,-122.475199,48.747436,NaN
2209,-122.463930,48.755411,NaN


In [ ]:
def get_closest_from_result(result):
    filtered = pd.DataFrame(result['searchResults'])
    closest = filtered.iloc[filtered['distance'].argmin()]
    return closest['distance'], closest['name'], closest['shapePoints'] 

In [ ]:
gas_stations_all.iloc[0]

Longitude             -122.329806
Latitude                 48.39376
name         24 Hour Express Fuel
Name: 0, dtype: object

In [ ]:
# for each gas station near I-5, compute its distance to closest exit
gas_station_attr_data = []
for gas_i in range(gas_stations_all.shape[0]):
  gas_data = gas_stations_all.iloc[gas_i]
  gas_origin = [gas_data['Latitude'], gas_data['Longitude']]
  # 1 mile radius attractions count
  result_r1 = post_radius_search_request(gas_origin, attractionRomoteDataList, 1)
  attr_cnt_1m = result_r1['resultsCount']

  # 5 miles radius attractions count
  result_r5 = post_radius_search_request(gas_origin, attractionRomoteDataList, 5)
  attr_cnt_5m = result_r5['resultsCount']
  # print(result_r5)

  # closest attraction name and loc and distance
  dist = np.nan
  name = np.nan
  attr_p = [np.nan, np.nan]
  if attr_cnt_5m > 0:
    dist, name, attr_p = get_closest_from_result(result_r5)
  
  gas_station_attr_data.append([gas_i, gas_data['name'], gas_data['Latitude'],gas_data['Longitude'],
                                attr_cnt_1m, attr_cnt_5m, 
                                name,attr_p[0],attr_p[1],dist])

In [ ]:
gas_station_attr_data = pd.DataFrame(gas_station_attr_data)
gas_station_attr_data.columns = ['gas_key', 'gas_name', 'gas_lat','gas_long',
                        'attr_cnt_1mile', 'attr_cnt_5mile',
                        'attr_name','attr_lat','attr_long','distance_to_nearest_attr']

In [ ]:
gas_station_attr_data

,gas_key,gas_name,gas_lat,gas_long,attr_cnt_1mile,attr_cnt_5mile,attr_name,attr_lat,attr_long,distance_to_nearest_attr
0,0,24 Hour Express Fuel,48.393760,-122.329806,0,1,Skagit County Fairgrounds,48.408145,-122.340118,1.1020
1,1,76,47.191712,-120.916225,0,1,KBHR Set,47.223241,-120.993224,4.2247
2,2,76,47.238704,-122.357577,0,4,Wagon Trail,47.275437,-122.308897,3.4174
3,3,76,48.435227,-122.291324,0,1,Skagit County Fairgrounds,48.408145,-122.340118,2.9200
4,4,76,47.839608,-120.014523,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2206,2206,NaN,48.198460,-122.127847,0,1,Giant Arborvitae (Big Cedar Stump),48.168640,-122.189179,3.5008
2207,2207,NaN,48.556192,-122.344545,0,0,NaN,NaN,NaN,NaN
2208,2208,NaN,48.747436,-122.475199,1,1,Mindport Exhibits,48.750881,-122.480674,0.3452
2209,2209,NaN,48.755411,-122.463930,1,1,Mindport Exhibits,48.750881,-122.480674,0.8254


In [ ]:
gas_station_attr_data.to_csv('Data/gas_station_w_attr.csv')